In [10]:
import os
import os
import re
import logging

from typing import Optional, Any
from typing import Any, Dict, Callable, List
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.output_parsers import JsonOutputParser

In [11]:
llm=AzureChatOpenAI(
azure_deployment =  os.getenv("CHAT_MODEL"),
azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT"),
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY"),
openai_api_version =  os.getenv("OPENAI_API_VERSION")
)

In [12]:
from typing_extensions import TypedDict
class State(TypedDict):
    question:str
    messages:str
    page_url:str
    frontend_origin: str
    generation:str
    route_to: Optional[str]


from langgraph.graph import StateGraph, END


In [ ]:
from langchain_core.prompts import ChatPromptTemplate


def route_user_query(State)->State:
    print("routing user query")
    prompt=ChatPromptTemplate.from_template("""You are a **smart routing assistant** for Spinnaker Analytics' LLM Agent Application.  
Your job is to determine if the user's provided `page_url` correctly matches one of the Spinnaker product catalog URLs and return a JSON object following the schema below.

---

### Output Schema
{{
    "is_correct_location": true or false,
    "is_incorrect_location_msg": "string or null",
    "route_to": "agent" or "llm_fallback" or "redirect"
}}

---
                                                        
## Rules to Follow
- If page_url matches a product catalog URL Or matches the Sub URL (If available) →
    "is_correct_location": true
    "is_incorrect_location_msg": null
    "route_to": "agent"

- If it does NOT match →
    "is_correct_location": false
    "is_incorrect_location_msg": "Format: To [briefly explain what they can do], please visit [Product Name] at {{frontend_origin}}/<product url>"
    "route_to": "redirect"

- General Queries and Greetings  →
    "is_correct_location": true
    "is_incorrect_location_msg": null
    "route_to": "llm_fallback"
 
--- 
   
## PRODUCT CATALOG:

### 1. MOG (Market Opportunity Generator / Market Intelligence)
  - **URL:** `/sales-prophet/individual-life/market-overview` 
  - **Sub URL:** [`sales-prophet/individual-life/wallet-share-assessment`, `sales-prophet/individual-life/sales-opportunity`, `sales-prophet/individual-life/agent-performance`]
  - **Purpose:** Estimate market size, analyze client performance, sales opportunity
  - **Query Keywords:** market premiums, client premiums/policies, agents, MSAs, states, counties, population, sales opportunity, carrier policies, effectiveness, premium share

### 2. Commission Intelligence (Contingent Commission)
  - **URL:** `/commission-intelligence`   
  - **Sub URL:** [`commission-intelligence/property-and-casualty/contract-ingestion`, `commission-intelligence/property-and-casualty/contract-summary`, `commission-intelligence/property-and-casualty/contract-comparison`]
  - **Purpose:** AI contract platform for analyzing commission structures
  - **Query Keywords:** contingent commission, carrier contracts, loss ratio, eligible written premium (EWP), growth rate, thresholds, contract comparison, Document Ingestion

### 3. Contract Comparator (aka Contingent Commission Contract Comparator)
  - **URL:** `/commission-intelligence/property-and-casualty/contract-comparison` 
  - **Sub URL:** `/commission-intelligence`
  - **Keywords:** compare contracts, side by side comparison, contract differences, better commission, carrier comparison, bonus structure comparison, eligibility comparison, commission rates comparison, which contract is better, contract analysis
  - **Description:** Side-by-side comparison tool for multiple contingent commission contracts across different carriers, highlighting key differences and similarities in commission structures, bonus types, eligibility criteria, thresholds, and requirements 

### 4. Contract Summary (aka Contingent Commission Contract Summary)
  - **URL:** `/commission-intelligence/property-and-casualty/contract-summary`
  - **Sub URL:** `/commission-intelligence`
  - **Keywords:** contracts summary, contract information, contract details, contract synopsis, contract abstract, contract condensation
  - **Description:** Centralized repository of all ingested contracts for easy search, filter and management. 

### 5. Spinnaker General Information
  - **Purpose:** Information about Spinnaker products, solutions, descriptions
  - **Query Keywords:** "what is", "tell me about", product features, demos, purchasing

---

### Routing Logic

1. **First, check for greetings or general questions:**
   - If user query is a greeting (hello, hi, how are you) or general question → Return `is_correct_location: true`, `is_incorrect_location_msg: null`, and `route_to: "llm_fallback"`

2. **Check if URL matches exactly:**
   - Compare the provided page_url with the URLs in the catalog
   - If the URL matches (also check if it's in Sub URL) → Return `is_correct_location: true`, `is_incorrect_location_msg: null`, and `route_to: "agent"`

3. **Special Commission Intelligence Rule:**
   - **CRITICAL:** If the page_url contains `/commission-intelligence` (anywhere in the path):
     - Read and understand the user query
     - Check if query keywords match Commission Intelligence, Contract Comparator, or Contract Summary keywords
     - If YES → Return `is_correct_location: true`, `is_incorrect_location_msg: null`, and `route_to: "agent"`
     - If NO (query is about a different product like MOG) → Go to step 4

4. **Infer product from user query:**
   - If no match yet, analyze the user query keywords against the product catalog
   - Match query to the most appropriate product based on keywords and description
   
   **IF the query clearly matches a product:**
   - Compare inferred product URL with current page_url
   - If they DON'T match:
     - Generate a contextual, friendly redirection message:
       - Acknowledge what the user is asking about
       - Briefly explain why they should visit the specific product page (1 sentence max)
       - Provide the redirect link using ONLY the exact URL from the product catalog
     - Format: "To [briefly explain what they can do], please visit [Product Name] at {{frontend_origin}}/<exact-product-url-from-catalog>"
     - Return `is_correct_location: false`, the message, and `route_to: "redirect"`
   
   **IF the query does NOT match any product in the catalog:**
   - Respond politely without providing any URL
   - Return `is_correct_location: true`, `is_incorrect_location_msg: null`, and `route_to: "llm_fallback"`
   
   **IF uncertain or confused:**
   - Return `is_correct_location: true`, `is_incorrect_location_msg: null`, and `route_to: "llm_fallback"`

5. **CRITICAL RULES:**
   - Only provide URLs that exist in the product catalog
   - If unsure which product matches, do NOT guess a URL
   - When in doubt, return `is_correct_location: true` and `route_to: "llm_fallback"`
   - Always return only valid JSON following the schema above
   - `is_incorrect_location_msg` should be proper Markdown text

---

## Please Find Below User Provided Input
User Question: {question}
Page URL: {page_url}
Frontend Origin: {frontend_origin}

Previous Conversation: 
{messages}

Return ONLY valid JSON with no additional text.
"""
    )
    
    question=State["question"]
    messages=State["messages"]
    page_url = State["page_url"]
    frontend_origin=State["frontend_origin"]
    
    json_parser = JsonOutputParser()
    question_router_chain = prompt | llm | json_parser
    
    try:
        response = question_router_chain.invoke({
            "question": question,
            "page_url": page_url,
            "messages": messages,
            "frontend_origin": frontend_origin
        })
        print(f"Response: {response}")
        route_to = response.get("route_to", "llm_fallback")
        State["route_to"] = route_to
        
        generation = response.get("is_incorrect_location_msg", "")
        State["generation"] = generation
        
    except Exception as e:
        logging.error(f"Error in route_user_query: {e}")
        State["route_to"] = "llm_fallback"
        State["generation"] = ""
    
    return State




In [ ]:
from datetime import datetime


def llm_fallback(State)->State:
    print("went to llm fallback")
    prompt=ChatPromptTemplate.from_template("""# Sage Chatbot — Persona & Experience

Below are core details about Sage’s persona, background, and capabilities.

---

## 1. What is your name?

**Q:** What should I call you?  
**A:** I’m **Sage**, the AI assistant for Spinnaker Analytics.

---

## 2. Who are you?
**A:** I’m **Sage**, the AI assistant for Spinnaker Analytics.

**Q:** Who is Sage?  
**A:** I’m the conversational interface for Spinnaker Analytics, here to help with questions and guidance.

---

## 3. What is your knowledge cutoff?

**Q:** How current is your information?  
**A:** My training data goes up to **June 2024**. For events or developments after that, I may need you to provide context.

---

## 4. How many years of experience do you have?

**Q:** How long has your team been practicing?  
**A:** The Spinnaker Analytics team averages **15+ years** of domain expertise, and the firm has **20+ years** of cumulative industry experience.

---

## 5. Are you available around the clock?

**Q:** Can I ask you questions at any time?  
**A:** Yes—Sage is available **24/7** to respond to your queries.

---

## 6. What is your response style?

**Q:** How will you answer my questions?  
**A:** I provide **clear, detailed, and actionable** responses—especially step‑by‑step guidance or code examples you can replicate directly.

</context>
When answer to user:
1. Answer the question as truthfully as possible from the context given to you. Do not try to make up any answer if you are not sure about it. If you’re uncertain about a topic, you should reply, "’ I’m not sure about that question, please reach out to info@spinnakeranalytics.com for more information".
2. Do not disclose any information of the spinnaker employees, client names,  CEOs, Team or Leadership or any personal information(except email address: info@spinnakeranalytics.com and phone number: +1 617-303-1937.), price of products or solutions.
3. Do not answer any question related to career or job openings or finance figures, sales figures etc. or any such information that is not available in the context and do not ask for any personal information from the user.
4. If question is asked regarding the demo or buying the product/solutions redirect them towards spinnaker analytics contact-us page (https://www.spinnakeranalytics.com/contact) or request-demo (https://www.spinnakeranalytics.com/?requestDemo=true).
5. Your final answer should be visually appealing for that you can use markdown/bullets/highlight the important information as you see fit.


\n\nCurrent time: {time}
\n\nPrevious messages: {messages}
\n\nQuestion: {question}

"""
    )
    
    question=State["question"]
    messages=State["messages"]
    time= datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    llm_fallback_chain= prompt|llm
    
    try:
        response = llm_fallback_chain.invoke({
            "question": question,
            "messages": messages,
            "time": time
        })
        
        # Extract content from response,it always returns an object
        if hasattr(response, 'content'):
            generation = response.content
        else:
            generation = str(response)
            
        State["generation"] = generation
        
    except Exception as e:
        logging.error(f"Error in llm_fallback: {e}")
        State["generation"] = "I'm having trouble processing your request. Please try again or contact info@spinnakeranalytics.com for assistance."
    
    return State

In [ ]:
def route_condition(state: State) -> str:
    """Determines which node to route to based on route_to value."""
    route_to = state.get("route_to", "llm_fallback")
    
    if route_to == "agent":
        return "call_agent"
    elif route_to == "redirect":
        return "handle_redirect"
    else:  # llm_fallback or any other value
        return "llm_fallback"


In [ ]:
def handle_redirect(state: State) -> State:
    """Handles redirection messages when user is on wrong page."""
    print("handling redirect")
    # The generation already contains the redirect message from route_user_query
    return state



In [ ]:

def call_your_agent(state: State) -> State:
    """Calls the main agent for product-specific queries."""
    print("calling agent")
    # TODO: Implement your agent logic here
    state["generation"] = "Agent functionality to be implemented."
    return state

In [ ]:
# Build the workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("route_query", route_user_query)
workflow.add_node("handle_redirect", handle_redirect)
workflow.add_node("llm_fallback", llm_fallback)
workflow.add_node("call_agent", call_your_agent)

# Set entry point
workflow.set_entry_point("route_query")

# Add conditional edges
workflow.add_conditional_edges(
    "route_query",
    route_condition,
    {
        "call_agent": "call_agent",
        "handle_redirect": "handle_redirect",
        "llm_fallback": "llm_fallback"
    }
)

# Add edges to END
workflow.add_edge("handle_redirect", END)
workflow.add_edge("llm_fallback", END)
workflow.add_edge("call_agent", END)

# Compile the graph
app = workflow.compile()


if __name__ == "__main__":
    # Test the workflow
    initial_state = {
   "question": "which is the best contract",
    "messages": "",
    "page_url": "/sales-prophet/individual-life/market-overview",
    "frontend_origin": "https://app.spinnakeranalytics.com",
    "generation": "",
    "route_to": None
    }
    
    result = app.invoke(initial_state)
    print("\n=== Result ===")
    print(f"Route: {result['route_to']}")
    print(f"Generation: {result['generation']}")

routing user query
handling redirect

=== Result ===
Route: redirect
Generation: To compare which contract is better, please visit Contract Comparator at https://app.spinnakeranalytics.com/commission-intelligence/property-and-casualty/contract-comparison
